In [2]:
import sys, os
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

sys.path.append('/content/drive/My Drive/Radiomics Workshop')
os.chdir('drive/My Drive/Radiomics Workshop')

Mounted at /content/drive


In [6]:
from mice_dataset import PatchMiceDatasetFromTensor
from torch.utils.data import DataLoader

bs = 1

dataset_name = 'dataForNet_shuffle_by_slices_2d_600_30per'
test_dataset = PatchMiceDatasetFromTensor(f'datasets/{dataset_name}', is_train=False)
dl_test = DataLoader(test_dataset, batch_size=bs, shuffle=False)

print(len(dl_test))

5586


In [ ]:
import numpy as np
import tqdm
import torch
from network import Classifier as PreTrained

model_path = "results/models/classification/classification_2d_new_data_new_net_wd"
model_name = "model_12_epochs"

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
use_location = False
print(device)
params = torch.load(f"{model_path}/{model_name}", map_location=device)
train_params = params['train_params']
model_state = params['model_state']
model = PreTrained(**train_params).to(device)
model.load_state_dict(model_state)
model.to(device)
print(model)

cuda


RuntimeError: ignored

In [5]:
tp = 0
tn = 0
fp = 0
fn = 0
for sample in dl_test:
  prediction = PreTrained(sample['image'])
  print(prediction)
  break

NameError: ignored

In [ ]:
import pandas as pd 

features = None
metadata_columns = ["Name", 'Slices', 'Patch', 'Tumor %', 'Path']
metadata = {name :[] for name in metadata_columns}

for batch in dl_test:
  if batch['label'] == 0:
    continue
  metadata["Name"].extend(batch['mouse_name'])
  metadata["Slices"].append(f"({batch['slice_range'][0].item()}, {batch['slice_range'][1].item()})")
  metadata["Patch"].append(f"({batch['patch'][0].item()}, {batch['patch'][1].item()})")
  metadata["Tumor %"].append(batch['tumor_percentage'].item())
  metadata["Path"].append(batch['path'])
  image = batch['image'].reshape([1,1,50,50]).to(device)
  batch_features = torch.flatten(model.extract_features(image)).reshape((1, 2304))
  if features is None:
    features = batch_features
  else:
    features = torch.cat((features, batch_features))

features = features.tolist()

meta_df = pd.DataFrame(metadata)
meta_df["id"] = meta_df.index
features_df = pd.DataFrame(features, columns=[f"Feature {i}" for i in range(len(features[0]))]) 
features_df["id"] = features_df.index
final_df = pd.merge(meta_df, features_df, on="id").drop("id", axis=1)


In [ ]:
final_df

,Name,Slices,Patch,Tumor %,Path,Feature 0,Feature 1,Feature 2,Feature 3,Feature 4,Feature 5,Feature 6,Feature 7,Feature 8,Feature 9,Feature 10,Feature 11,Feature 12,Feature 13,Feature 14,Feature 15,Feature 16,Feature 17,Feature 18,Feature 19,Feature 20,Feature 21,Feature 22,Feature 23,Feature 24,Feature 25,Feature 26,Feature 27,Feature 28,Feature 29,Feature 30,Feature 31,Feature 32,Feature 33,Feature 34,...,Feature 2264,Feature 2265,Feature 2266,Feature 2267,Feature 2268,Feature 2269,Feature 2270,Feature 2271,Feature 2272,Feature 2273,Feature 2274,Feature 2275,Feature 2276,Feature 2277,Feature 2278,Feature 2279,Feature 2280,Feature 2281,Feature 2282,Feature 2283,Feature 2284,Feature 2285,Feature 2286,Feature 2287,Feature 2288,Feature 2289,Feature 2290,Feature 2291,Feature 2292,Feature 2293,Feature 2294,Feature 2295,Feature 2296,Feature 2297,Feature 2298,Feature 2299,Feature 2300,Feature 2301,Feature 2302,Feature 2303
0,MB1333220714F000H00000000C000000,"(153, 153)","(3, 1)",0.4440,[datasets/dataForNet_shuffle_by_slices_2d_600_...,0.154798,0.418654,0.000000,0.000000,0.0,1.920056,0.184496,0.575891,0.0,0.000000,0.000000,1.428783,0.379931,0.740044,0.000000,0.000000,0.349286,1.904221,0.246579,0.019210,0.0,0.000000,0.0,0.228594,0.405143,0.000000,0.0,0.0,0.0,0.155495,1.483792,1.316087,0.485826,0.480459,0.000000,...,0.292264,0.000000,0.000000,0.000000,0.000000,0.000000,0.186894,1.354444,0.963194,0.000000,0.000000,0.000000,0.416972,1.736655,0.372334,0.000000,1.575462,0.000000,0.000000,0.379810,0.904163,0.000000,0.595497,0.19481,0.869893,0.000000,0.0,0.000000,0.0,0.0,0.041790,0.000000,0.492235,0.000000,0.134525,0.429571,0.363724,0.000000,0.047972,0.479967
1,MB1333220714F000H00000000C000000,"(154, 154)","(2, 0)",0.5496,[datasets/dataForNet_shuffle_by_slices_2d_600_...,0.149220,0.639444,0.000000,0.912427,0.0,0.568772,0.521725,0.497201,0.0,2.855080,0.000000,0.000000,0.000000,1.453345,1.071215,1.667773,0.000000,0.000000,0.000000,0.298475,0.0,0.000000,0.0,0.000000,0.000000,0.111969,0.0,0.0,0.0,0.000000,0.233091,0.618761,2.022151,0.292539,0.000000,...,0.150019,0.000000,0.000000,0.521280,0.449021,0.000000,1.244119,0.852483,0.000000,0.000000,1.448190,0.109749,0.480638,0.000000,0.000000,0.831485,1.350266,0.000000,0.000000,0.000000,0.000000,0.000000,1.273666,0.00000,0.000000,0.000000,0.0,1.581657,0.0,0.0,0.000000,0.000000,0.000000,1.114818,0.187663,0.923431,0.000000,0.000000,0.000000,1.259697
2,MB1333220714F000H00000000C000000,"(154, 154)","(2, 1)",0.9172,[datasets/dataForNet_shuffle_by_slices_2d_600_...,1.276884,0.321434,0.000000,0.079446,0.0,1.207538,0.788025,0.515488,0.0,0.000000,0.000000,0.876837,1.114907,1.007746,0.000000,0.000000,0.000000,0.537323,1.051637,0.513538,0.0,0.000000,0.0,0.000000,0.416399,0.000000,0.0,0.0,0.0,0.000000,2.050883,0.958955,0.000000,0.113417,0.606387,...,0.037705,0.169729,0.000000,0.380865,0.000000,0.000000,0.270813,0.601752,0.393115,0.000000,0.569775,0.000000,0.000000,0.576106,0.070231,1.563843,1.645842,0.000000,0.000000,0.000000,0.000000,1.311455,0.000000,0.00000,0.230518,0.000000,0.0,0.524335,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.632956,0.618087,0.000000,0.228906,1.378437,1.566503
3,MB1333220714F000H00000000C000000,"(154, 154)","(3, 0)",0.3924,[datasets/dataForNet_shuffle_by_slices_2d_600_...,1.541377,0.517707,0.227365,0.105508,0.0,0.000000,1.484067,1.161290,0.0,0.478637,0.000000,0.000000,2.137905,0.731039,0.000000,0.079908,0.000000,0.000000,0.478035,0.485729,0.0,0.000000,0.0,0.244684,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.130461,1.773688,0.000000,0.617575,1.873073,...,0.059793,0.000000,0.000000,0.387413,1.256481,0.479248,0.424256,0.000000,0.285683,0.605802,2.078329,0.000000,0.178732,0.000000,0.000000,1.174001,0.000000,0.000000,1.473058,0.296111,0.000000,1.463991,0.000000,0.00000,1.258405,1.240875,0.0,0.661695,0.0,0.0,0.000000,0.173407,0.000000,0.000000,0.000000,0.000000,0.000000,0.846884,0.000000,0.085715
4,MB1333220714F000H00000000C000000,"(154, 154)","(3, 1)",0.4440,[datasets/dataForNet_shuffle_by_slice

In [ ]:
path_to_root = 'results/features'
file_name = 'segmentation_features'
final_df.to_csv(f'{path_to_root}/{file_name}_from_{model_name}.csv')